## Overview

This notebook visualizes that some data in "../input/smartphone-decimeter-2022/train/2020-08-03-US-MTV-2/GooglePixel5/device_gnss.csv" is not smooth.

The root of "ground_truth.csv" is smooth, but certain period of "device_gnss.csv" does not appear to be smooth.
To be precise, two consecutive positions within a certain period show the same position.

I don't think I have enough words to describe it, so please take a look at the figure.

I have shared this question in the discussion.

Thank you.

### Import lib

In [ ]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

### Visualize util

I referred to this notebook : [📱 Smartphone Competition 2022 [Twitch Stream]](https://www.kaggle.com/code/robikscube/smartphone-competition-2022-twitch-stream)

In [ ]:
def visualize_traffic(
    df,
    lat_col="LatitudeDegrees",
    lon_col="LongitudeDegrees",
    center=None,
    color_col="phone",
    label_col="tripId",
    zoom=9,
    opacity=1,
):
    if center is None:
        center = {
            "lat": df[lat_col].mean(),
            "lon": df[lon_col].mean(),
        }
    fig = px.scatter_mapbox(
        df,
        # Here, plotly gets, (x,y) coordinates
        lat=lat_col,
        lon=lon_col,
        # Here, plotly detects color of series
        color=color_col,
        labels=label_col,
        zoom=zoom,
        center=center,
        height=600,
        width=800,
        opacity=0.5,
    )
    fig.update_layout(mapbox_style="stamen-terrain")
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()

### Specify the display period

In [ ]:
st = 1596501799434
et = 1596501818434
time_width = 1000

n_labels = int((et-st)/time_width + 1)

## Visualize Ground truth

It is hard to see without hovering the mouse. Sorry.

In [ ]:
gt_df = pd.read_csv("../input/smartphone-decimeter-2022/train/2020-08-03-US-MTV-2/GooglePixel5/ground_truth.csv")
specific_time_gt_df = gt_df[(gt_df["UnixTimeMillis"] >= st) & (gt_df["UnixTimeMillis"] <= et)]
visualize_traffic(
    specific_time_gt_df,
    lat_col="LatitudeDegrees",
    lon_col="LongitudeDegrees",
    color_col="UnixTimeMillis",
    zoom=15,
)

## Visualize device_gnss.csv

In [ ]:
df = pd.read_csv("../input/smartphone-decimeter-2022/train/2020-08-03-US-MTV-2/GooglePixel5/device_gnss.csv")
cmap = [numpy.random.rand(3,) for _ in range(n_labels)]
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(25,8))
for ax_id, col in enumerate(["WlsPositionXEcefMeters", "WlsPositionYEcefMeters", "WlsPositionZEcefMeters"]):
    axes[ax_id].set_title(col)
    for idx, utcTime in enumerate(range(st, et, time_width)):
        specific_time_df = df[df["utcTimeMillis"]==utcTime]
        axes[ax_id].scatter(specific_time_df.index, specific_time_df.WlsPositionYEcefMeters, s=10, c=cmap[idx])
plt.legend(range(st, et, time_width))
plt.show()